## 1. Import packages

In [1]:
import glob
import time
import cv2
import torch
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO

In [2]:
!nvidia-smi

Fri Jan 16 16:27:58 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.95.05              Driver Version: 580.95.05      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060        Off |   00000000:01:00.0 Off |                  N/A |
| 54%   51C    P3             36W /  170W |     639MiB /  12288MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 2. Config

In [3]:
ONNX_PATH = "/home/mlops/Repository/aio2025-onnx-tensorrt/models/yolo26l.onnx"
IMAGE_GLOB = "/home/mlops/Repository/aio2025-onnx-tensorrt/images/val2014/*.jpg"
IMG_SIZE = 640
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
WARMUP = 20
SAMPLE_SIZE = 1000

## 3. Preprocess function

In [4]:
def preprocess(img, size=640):
    img = cv2.resize(img, (size, size))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = np.transpose(img, (2, 0, 1))
    img = np.expand_dims(img, 0)
    return torch.from_numpy(img)

# Load images
image_paths = sorted(glob.glob(IMAGE_GLOB))
assert len(image_paths) > 0, "No images found"
if SAMPLE_SIZE != -1:
    image_paths = image_paths[:SAMPLE_SIZE]

## 4. Model

In [5]:
# Load model
model = YOLO(ONNX_PATH, task='detect')

# Warmup
dummy = torch.zeros((1, 3, IMG_SIZE, IMG_SIZE), device=DEVICE)
for _ in range(WARMUP):
    _ = model(dummy, verbose=False)

Loading /home/mlops/Repository/aio2025-onnx-tensorrt/models/yolo26l.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.21.0 with CUDAExecutionProvider


2026-01-16 16:27:59.117041866 [W:onnxruntime:, transformer_memcpy.cc:83 ApplyImpl] 6 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.


## 5. Test loop

In [6]:
# FPS test
start = time.perf_counter()

for p in tqdm(image_paths):
    img = cv2.imread(p)
    inp = preprocess(img, IMG_SIZE).to(DEVICE)
    _ = model(inp, verbose=False)
end = time.perf_counter()

total_time = end - start
fps = len(image_paths) / total_time

100%|██████████| 1000/1000 [00:25<00:00, 38.93it/s]


In [7]:
print("=" * 40)
print(f"Device           : {DEVICE}")
print(f"Images processed : {len(image_paths)}")
print(f"Total time       : {total_time:.2f} s")
print(f"FPS              : {fps:.2f}")
print("=" * 40)

Device           : cuda
Images processed : 1000
Total time       : 25.69 s
FPS              : 38.93
